# Boilerplate

In [ ]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

init_logging()

# Run

List all Python wrappers

In [ ]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

## Transport layer setup

Create the Transport layer and run it in the background.

In [ ]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62014
local_notebook_server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")

client_tiface = await new_transport_interface("0.0.0.0", 0, TransportModeW("client"))
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT, TransportModeW("server"))
server_tiface.run()

# Filter layer setup

In [ ]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

Find methods to run

In [ ]:
[m for m in dir(client_fiface) if not m.startswith('__')]

## Run Client and Server Filter!

In [ ]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above futures should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

# Filter layer operation

## Change the server status to something meaningful

In [ ]:
server_name = "Local Conwayste Server (notebook)"
server_version = "1.2.34.56"
player_count = 8675309
room_count = 9001
await server_fiface.command_response(FilterCmdW("changeserverstatus",
                                                server_name=server_name,
                                                server_version=server_version,
                                                player_count=player_count,
                                                room_count=room_count))

## As a client, connect to server

### First, resolve DNS for Chococat

In [ ]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ept  = EndpointW(f"{choco_ip}:2016")
choco_ept

### Next create a "Connect" RequestAction

In [ ]:

import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect",
                                name="FilterNotebookClient{}".format(id),
                                client_version="0.0.666")
connect_action

### Now, actually send it!

**Helpful tip**: you can substitute `chococat_ept` for `local_notebook_server_ept` and vice versa. This controls whether you are talking to the official Conwayste server (Chococat) or a notebook-local one.

In [ ]:
# CHANGE THIS IF YOU WANT
server_ept = local_notebook_server_ept

# Just a convenience function to make all this easier
async def request_action_to_filter(req_action, endpoint=server_ept):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

await request_action_to_filter(connect_action)

## As a server, handle the incoming Connect

The notebook is acting as the server app layer here.

In [ ]:
time.sleep(0.1)

svr_notifs = server_fiface.get_notifications()
assert svr_notifs[0].variant == 'ClientAuthRequest'
auth_req = svr_notifs[0]
client_ept_from_connect = auth_req.endpoint
[client_ept_from_connect, auth_req]

Make a cookie and send it back in an `LoggedIn` response code to the client. The filter generates this upon submitting a `CompleteAuthRequest` filter action with a `LoggedIn` `AuthDecision`.

In [ ]:
cookie = 'FakeCookieNotReallyDelicious'

decision = AuthDecisionW('loggedin', cookie=cookie, server_version='0.0.667')

await server_fiface.command_response(FilterCmdW("completeauthrequest",
                                                endpoint=client_ept_from_connect,
                                                decision=decision))

## As a client, get notifications (the `LoggedIn` response code)

In [ ]:
time.sleep(0.05)
client_fiface.get_notifications()

In [ ]:
new_response_code = _[0]
new_response_code

Get cookie

In [ ]:
cookie = new_response_code.response_code.cookie
cookie

## As a server, get notifications

In [ ]:
time.sleep(0.05)
notifs = server_fiface.get_notifications()

## As a client, join a room

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

In [ ]:
# not expecting any
client_notifs = client_fiface.get_notifications()
print(client_notifs)
assert len(client_notifs) == 0

## As a server...

### Get notifications
Should be a JoinRoom

In [ ]:
time.sleep(0.05)
notifs = server_fiface.get_notifications()
notifs

### Send JoinedRoom

In [ ]:
code = ResponseCodeW("joinedroom", room_name="general") # ToDo: get room name from notifs
await server_fiface.command_response(FilterCmdW("sendresponsecode",
                                                endpoint=client_ept_from_connect,
                                                code=code))

### Send a GameUpdate

In [ ]:
game_updates = [GameUpdateW("gamenotification", msg="now we are playing for CASH MONEY ;)")]
await server_fiface.command_response(FilterCmdW("sendgameupdates",
                                                endpoints=[client_ept_from_connect], updates=game_updates))

## As a client...

### Get notifications

Expecting a joined room and a game notification

In [ ]:
time.sleep(0.05)
client_fiface.get_notifications()